In [ ]:
def solve_logistics_csp():
    packages = [1, 2, 3, 4, 5]

    cities = {'A': 2, 'B': 3, 'C': 1, 'D': 2}
    warehouses = {'W1': 3, 'W2': 4}
    all_locations = {**cities, **warehouses}

    routes = {
        1: [('W1', 'A'), ('W1', 'B'), ('A',), ('B',)],
        2: [('W1', 'C'), ('W1', 'D')],
        3: [('A',), ('D',), ('A', 'D')],
        4: [('W2', 'B')],
        5: [('W1',), ('W2',), ('C',), ('W2', 'C')]
    }

    priorities = {
        1: 'high',
        2: 'high',
        3: 'medium',
        4: 'medium',
        5: 'low'
    }

    route_costs = {
        ('W1', 'A'): 3,
        ('W1', 'B'): 2,
        ('W2', 'C'): 4,
        ('A', 'D'): 5
    }

    def calculate_route_cost(route):
        if len(route) < 2:
            return 0

        cost = 0
        for i in range(len(route) - 1):
            segment = (route[i], route[i+1])
            cost += route_costs.get(segment, 1)
        return cost

    def select_variable_mrv(assignment, domains):
        unassigned = [p for p in packages if p not in assignment]
        if not unassigned:
            return None

        rank = {'high': 2, 'medium': 1, 'low': 0}

        return min(unassigned, key=lambda p: (len(domains[p]), rank[priorities[p]]))

    def select_variable_degree(assignment, domains, constraints_map):
        unassigned = [p for p in packages if p not in assignment]
        if not unassigned:
            return None

        return max(unassigned, key=lambda p: sum(1 for q in unassigned if q != p and (p, q) in constraints_map))

    def order_values_lcv(package, domains, assignment, caps):
        def score(route):
            temp_caps = caps.copy()

            for loc in route:
                temp_caps[loc] -= 1

            eliminated = 0
            for other_pkg in packages:
                if other_pkg == package or other_pkg in assignment:
                    continue

                for other_route in domains[other_pkg]:
                    if any(temp_caps[loc] < 0 for loc in other_route):
                        eliminated += 1

            return (eliminated, calculate_route_cost(route))

        return sorted(domains[package], key=score)

    def is_consistent(route, caps):
        for loc in route:
            if caps[loc] - 1 < 0:
                return False
        return True

    def apply_arc_consistency(domains, caps):
        revised_domains = {p: list(d) for p, d in domains.items()}

        change = True
        while change:
            change = False
            for p in packages:
                consistent_routes = [r for r in revised_domains[p] if is_consistent(r, caps)]

                if not consistent_routes:
                    return None

                if len(consistent_routes) < len(revised_domains[p]):
                    revised_domains[p] = consistent_routes
                    change = True

        return revised_domains

    def backtrack(assign, domains, caps):
        if len(assign) == len(packages):
            return assign

        domains = apply_arc_consistency(domains, caps)
        if domains is None:
            return None

        pkg = select_variable_mrv(assign, domains)
        if pkg is None or not domains[pkg]:
            return None

        for route in order_values_lcv(pkg, domains, assign, caps):
            if is_consistent(route, caps):
                new_caps = caps.copy()
                for loc in route:
                    new_caps[loc] -= 1

                assign[pkg] = route
                result = backtrack(assign, domains, new_caps)
                if result:
                    return result

                del assign[pkg]

        return None

    constraints_map = {}
    for p1 in packages:
        for p2 in packages:
            if p1 != p2:
                p1_locs = set(loc for route in routes[p1] for loc in route)
                p2_locs = set(loc for route in routes[p2] for loc in route)
                if p1_locs.intersection(p2_locs):
                    constraints_map[(p1, p2)] = True
                    constraints_map[(p2, p1)] = True

    solution = backtrack({}, {p: routes[p][:] for p in packages}, all_locations.copy())

    total_cost = sum(calculate_route_cost(r) for r in solution.values()) if solution else 0

    return solution, total_cost

solution, total_cost = solve_logistics_csp()

if solution:
    print("\nPackage Assignment:")
    for p in sorted(solution):
        route = solution[p]

        route_cost = 0
        for i in range(len(route) - 1):
            segment = (route[i], route[i+1])
            route_cost += {'W1A': 3, 'W1B': 2, 'W2C': 4, 'AD': 5}.get(segment[0] + segment[1], 1)

        route_str = " -> ".join(route)

        print(f"Package {p}: {route_str} (Cost: {route_cost})")

    print(f"\nTotal Cost: {total_cost}")
else:
    print("No solution found.")